In [ ]:
%matplotlib inline
%tensorflow_version 1.x
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
tfd = tfp.distributions
import math
import os

TensorFlow 1.x selected.


In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
tfd = tfp.distributions
import math
import os

In [34]:
ber = tf.zeros((4),tf.float32)

In [35]:
ber

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [31]:
tf.complex([5.0],[1.0])[0]

<tf.Tensor: shape=(), dtype=complex64, numpy=(5+1j)>

In [37]:
ber_list = tf.unstack(ber)
ber_list[0] = tf.constant(5,tf.float32)
ber = tf.stack(ber_list)

In [38]:
ber

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([5., 0., 0., 0.], dtype=float32)>

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
import utils as cu

import tx as ctx
import autoencoder as ae
import helper as cfh
import training as cft

In [ ]:
print('Tensorflow version:', tf.__version__)
print('Tensorflow probability version:', tfp.__version__)

Tensorflow version: 1.15.2
Tensorflow probability version: 0.7.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class IQ_Imbalance():
    def __init__(self,alpha,theta,option):
        self.option = option
        self.alpha = alpha
        self.theta = theta
        
        if self.option == 1:
            mu_r = (1 + alpha*tf.cos(theta))/2
            mu_i = (alpha * tf.sin(theta))/2
            nu_r = (1 - alpha*tf.cos(theta))/2
            nu_i = (alpha*tf.sin(theta))/2
        else:
            mu_r = tf.cos(theta/2)
            mu_i = -alpha*tf.sin(theta/2)
            nu_r = alpha*tf.cos(theta/2)
            nu_i = tf.sin(theta/2)
        mu = tf.stack([mu_r,mu_i],0)
        nu = tf.stack([nu_r,nu_i],0)
        self.mu = cfh.real2complex(mu)
        self.nu = cfh.real2complex(nu)
    def __call__(self,x):
        x_complex = cfh.real2complex(x)
        # x_complex = tf.convert_to_tensor(x_complex.numpy())
        x_o = self.mu*x_complex + self.nu*tf.conj(x_complex)
        x_o = cfh.complex2real(x_o)
        return x_o

In [ ]:
seed = 42
tf.set_random_seed(seed)
np.random.seed(seed)

# Parameters
# Channel Parameters
chParam = cu.AttrDict()
chParam.M = 64
chParam.SNR_db = 30

# Auto-Encoder Parameters
aeParam = cu.AttrDict()
aeParam.constellationDim = 2
aeParam.constellationOrder = chParam.M
aeParam.temperature = 1
aeParam.nLayersEnc  = 1
aeParam.nLayersDec  = 2
aeParam.nHiddenEnc  = 128
aeParam.nHiddenDec  = 128
aeParam.activation  = tf.nn.relu
aeParam.dtype       = tf.float32
aeParam.cpx_dtype   = tf.complex64

# Training Parameters
trainingParam = cu.AttrDict()
trainingParam.nBatches      = 16
trainingParam.batchSize     = 32*chParam.M
trainingParam.learningRate  = 0.001
trainingParam.displayStep   = 5
trainingParam.iterations    = 70

# IQ Parameters
alpha1 = tf.constant(0.9)
theta1 = tf.constant(np.pi*0.3)
alpha2 = tf.constant(0.8)
theta2 = tf.constant(-np.pi*0.2)

In [ ]:
# TF constants
one = tf.constant(1, aeParam.dtype)
twoZeroCpx = tf.constant(2, aeParam.cpx_dtype)

In [ ]:
def p_norm(p, x, fun=lambda x: tf.square(tf.abs(x))):
    return tf.reduce_sum(p * fun(x))

def r2c(x):
    return tf.cast(x, aeParam.cpx_dtype)

def c2r(x):
    return tf.cast(x, aeParam.dtype)

@tf.custom_gradient
def straight_through_estimator(s_bar):
    s = tf.one_hot(tf.math.argmax(s_bar, axis=-1), chParam.M)
    def grad(dy):
        return dy
    return s, grad

In [ ]:
SNR_db = tf.placeholder(aeParam.dtype, shape=(1,1))

# Sampling
temperature = tf.constant(aeParam.temperature, aeParam.dtype)
enc_inp = tf.constant(1, dtype=aeParam.dtype, shape=(1,1))
s_logits = ae._encoder(enc_inp, aeParam.nHiddenEnc, aeParam.nLayersEnc, aeParam.activation, nOutput=chParam.M, name='encoder')
g_dist = tfd.Gumbel(loc=0., scale=1.)
g = g_dist.sample(sample_shape=[trainingParam.batchSize, chParam.M])
s_bar = tf.nn.softmax((g + s_logits)/temperature)

# straight through estimator
s = straight_through_estimator(s_bar)

In [ ]:
# geo shaping
enc_vec = ae._encoder(s, aeParam.nHiddenEnc, aeParam.nLayersEnc, aeParam.activation, nOutput=aeParam.constellationDim, name='encoder_geo')
xSeed = tf.linalg.eye(aeParam.constellationOrder, dtype=s.dtype)
constellation_vec = ae._encoder(xSeed, aeParam.nHiddenEnc, aeParam.nLayersEnc, aeParam.activation, nOutput=aeParam.constellationDim, name='encoder_geo')
constellation = tf.expand_dims(tf.complex(constellation_vec[:, 0], constellation_vec[:, 1]), 0)
enc = tf.expand_dims(tf.complex(enc_vec[:, 0], enc_vec[:, 1]), -1)

In [ ]:
# normalization & Modulation
p_s = tf.nn.softmax(s_logits)
norm_factor = tf.rsqrt(p_norm(p_s, constellation))
norm_constellation = r2c(norm_factor) * constellation
x = r2c(norm_factor) * enc

In [ ]:
def PA_nonlinearity(x):
    x_norm = 2
    a_t = tf.math.abs(x)/x_norm
    # PA non-lineaerity parameters
    alpha_A = tf.constant(2.996048, tf.float32)
    beta_A = tf.constant(0.515222, tf.float32)
    alpha_p = tf.constant(5.544818e-14, tf.float32)
    beta_p = tf.constant(141.637485, tf.float32)
    complex_i = tf.constant(1j,dtype=tf.complex64)
    normal_factor = alpha_A/(1+beta_A)
    
    C_a_t = alpha_A/(1+beta_A*tf.math.square(a_t))/normal_factor
    phi_a_t = alpha_p*tf.math.square(a_t)/(1+beta_p*tf.math.square(a_t))
    C_a_t = tf.complex(tf.math.cos(phi_a_t)*C_a_t, tf.math.sin(phi_a_t)*C_a_t)
    y_t = tf.multiply(C_a_t,x)
    return y_t

In [ ]:
# def PA_nonlinearity_tanh(alpha,x):
#   # return tf.math.sigmoid(alpha * x)
#   return tf.math.tan(x)

In [ ]:
x = PA_nonlinearity(x)

In [ ]:
# checks
should_always_be_one = p_norm(p_s, norm_constellation)

# Channel
noise_cpx = tf.complex(tf.random_normal(shape=tf.shape(x), dtype=aeParam.dtype),
                       tf.random_normal(shape=tf.shape(x), dtype=aeParam.dtype))
sigma2 = one / cfh.dB2lin(SNR_db, 'dB')
noise_snr = r2c(tf.sqrt(sigma2)) * tf.rsqrt(twoZeroCpx) * noise_cpx
y = x + noise_snr


In [ ]:
# demodulator1
y_vec1 = cfh.complex2real(tf.squeeze(y))
print(y_vec1)
IQ1 = IQ_Imbalance(alpha1,theta1,1)
y_vec1 = IQ1(y_vec1)
dec1 = ae._encoder(y_vec1, aeParam.nHiddenDec, aeParam.nLayersDec, aeParam.activation, nOutput=chParam.M, name='decoder')

Tensor("stack:0", shape=(2048, 2), dtype=float32)


In [ ]:
# demodulator2
y_vec2 = cfh.complex2real(tf.squeeze(y))
IQ2 = IQ_Imbalance(alpha2,theta2,1)
y_vec2 = IQ2(y_vec2)
dec2 = ae._encoder(y_vec2, aeParam.nHiddenDec, aeParam.nLayersDec, aeParam.activation, nOutput=chParam.M, name='decoder')

In [ ]:
y_vec1.shape

TensorShape([Dimension(2048), Dimension(2)])

In [ ]:
y_vec2.shape

TensorShape([Dimension(2048), Dimension(2)])

In [ ]:
s.shape

TensorShape([Dimension(2048), Dimension(64)])

In [ ]:
# loss
loss1 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(s), logits=dec1)
entropy_S1 = -p_norm(p_s, p_s, lambda x: cfh.log2(x))
loss_hat1 = loss1 - entropy_S1
learning_rate1 = tf.placeholder(tf.float32)
gaussian_MI1 = cfh.gaussianMI(x, y, norm_constellation, chParam.M, dtype=aeParam.dtype) # change from x to PA_nonlinearity(x)

optimizer1 = tf.train.AdamOptimizer(learning_rate=learning_rate1).minimize(loss_hat1)

metricsDict1 = {'loss_hat': loss_hat1, 'gaussian_MI_metric': gaussian_MI1}
meanMetricOpsDict1, updateOps1, resetOps1 = cft.create_mean_metrics(metricsDict1)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
updateOps1

[<tf.Tensor 'loss_hat/mean/update_op:0' shape=() dtype=float32>,
 <tf.Tensor 'gaussian_MI_metric/mean/update_op:0' shape=() dtype=float32>]

In [ ]:
# loss
loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(s), logits=dec2)
entropy_S2 = -p_norm(p_s, p_s, lambda x: cfh.log2(x))
loss_hat2 = loss2 - entropy_S2
learning_rate2 = tf.placeholder(tf.float32)
gaussian_MI2 = cfh.gaussianMI(x, y, norm_constellation, chParam.M, dtype=aeParam.dtype)

optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate2).minimize(loss_hat2)

metricsDict2 = {'loss_hat': loss_hat2, 'gaussian_MI_metric': gaussian_MI2}
meanMetricOpsDict2, updateOps2, resetOps2 = cft.create_mean_metrics(metricsDict2)

In [ ]:
updateOps2

[<tf.Tensor 'loss_hat_1/mean/update_op:0' shape=() dtype=float32>,
 <tf.Tensor 'gaussian_MI_metric_1/mean/update_op:0' shape=() dtype=float32>]

In [ ]:
# loss
# loss = loss1 + loss2
alpha_loss = tf.placeholder(tf.float32)
loss = alpha_loss*loss_hat1+(1-alpha_loss) * loss_hat2
learning_rate = tf.placeholder(tf.float32)
gaussian_MI = cfh.gaussianMI(x, y, norm_constellation, chParam.M, dtype=aeParam.dtype)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
metricsDict = {'loss_hat': loss, 'gaussian_MI_metric': gaussian_MI}
meanMetricOpsDict, updateOps, resetOps = cft.create_mean_metrics(metricsDict)

In [ ]:
meanMetricOpsDict

{'gaussian_MI_metric': <tf.Tensor 'gaussian_MI_metric_2/mean/value:0' shape=() dtype=float32>,
 'loss_hat': <tf.Tensor 'loss_hat_2/mean/value:0' shape=() dtype=float32>}

In [ ]:
updateOps

[<tf.Tensor 'loss_hat_2/mean/update_op:0' shape=() dtype=float32>,
 <tf.Tensor 'gaussian_MI_metric_2/mean/update_op:0' shape=() dtype=float32>]

Each time gradient store

In [ ]:
# init = tf.global_variables_initializer()
# sess = tf.Session(config=config)
# sess.run(init)

In [ ]:
def save_data(epoch):
  np_x = []
  np_y = []
  for _ in range(1000):
      temp_x, temp_y = sess.run([x, y], feed_dict=feedDict)
      np_x.append(temp_x)
      np_y.append(temp_y)
  all_y = np.reshape(np.stack(np_y),-1)
  all_x = np.reshape(np.stack(np_x),-1)

  noise = np.random.normal(0,1,size=all_x.shape)+1j*np.random.normal(0,1,size=all_x.shape) 
  all_x = all_x + 0.05*noise
  np.save('/content/drive/MyDrive/colab/save_data/x_value_'+str(epoch)+'.npy',all_x)
  # np.save('/content/drive/MyDrive/colab/save_data/y_value_'+str(epoch)+'.npy',all_y)
  print("saved")

In [ ]:
chParam.SNR_db = np.arange(0,30.1,0.1)

In [ ]:
# np_SNR_db = chParam.SNR_db * np.ones((1,1))

Two decoders as turn

In [ ]:
# # np_SNR_db = chParam.SNR_db * np.ones((1,1))
# count = 0
# for i in chParam.SNR_db:
#   init = tf.global_variables_initializer()# zero gradient
#   sess = tf.Session(config=config)
#   sess.run(init)
#   loss1_now = loss_hat1
#   loss2_now = loss_hat2
#   alpha_loss_temp = 0.5
#   print("snr %f" % i)
#   np_SNR_db = np.round(i,1) * np.ones((1,1))
#   for epoch in range(1, trainingParam.iterations+1):
#       if epoch % 2 == 0:

#         # sess.run(resetOps1)
#         sess.run(resetOps)
#         # batch_progressive = [int(i) for i in np.linspace(100,10000,trainingParam.iterations)]
#         # lr_progressive = [i for i in np.linspace(1e-3,1e-5,trainingParam.iterations)]
#         for batch in range(0,trainingParam.nBatches):
#         # for batch in range(0,batch_progressive[epoch-1]):
#             # feedDict = {SNR_db: np_SNR_db,learning_rate1: lr_progressive[epoch-1]}
#             feedDict = {SNR_db: np_SNR_db,learning_rate:0.001,alpha_loss: alpha_loss_temp}
#             # feedDict1 = {SNR_db: np_SNR_db,learning_rate1:0.001}
#             sess.run([optimizer, updateOps], feed_dict=feedDict)
            
#             # sess.run([optimizer1, updateOps1], feed_dict=feedDict1)
#         [outLossHat, outGausianMI, outShouldAlwaysBeOne] = sess.run([meanMetricOpsDict['loss_hat'],
#                                                       meanMetricOpsDict['gaussian_MI_metric'],
#                                                       should_always_be_one], feed_dict=feedDict)
#         loss1_now = sess.run(loss_hat1,feed_dict=feedDict)
#         loss2_now = sess.run(loss_hat2,feed_dict=feedDict)
#         loss1_now = np.sum(loss1_now)/trainingParam.nBatches
#         loss2_now = np.sum(loss2_now)/trainingParam.nBatches
#         alpha_loss_temp = loss1_now/(loss1_now + loss2_now)
#         print('IQ one epoch: {} - lossHat: {:.3} - gaussianMI: {:.3} - always 1: {:.2}'.format(epoch, outLossHat, outGausianMI, outShouldAlwaysBeOne))
#       else:
#         # sess.run(resetOps2)
#         sess.run(resetOps)
#         # batch_progressive = [int(i) for i in np.linspace(100,10000,trainingParam.iterations)]
#         # lr_progressive = [i for i in np.linspace(1e-3,1e-5,trainingParam.iterations)]
#         for batch in range(0,trainingParam.nBatches):
#         # for batch in range(0,batch_progressive[epoch-1]):
#             # feedDict = {SNR_db: np_SNR_db,learning_rate2: lr_progressive[epoch-1]}
#             feedDict = {SNR_db: np_SNR_db,learning_rate:0.001,alpha_loss: alpha_loss_temp}
#             # feedDict2 = {SNR_db: np_SNR_db,learning_rate2:0.001}
#             sess.run([optimizer, updateOps], feed_dict=feedDict)
#             # sess.run([optimizer2, updateOps2], feed_dict=feedDict2)
#         # outx,outy,con = sess.run([x,y,norm_constellation], feed_dict=feedDict)
#         [outLossHat, outGausianMI, outShouldAlwaysBeOne] = sess.run([meanMetricOpsDict['loss_hat'],
#                                                       meanMetricOpsDict['gaussian_MI_metric'],
#                                                       should_always_be_one], feed_dict=feedDict)
#         print('IQ two epoch: {} - lossHat: {:.3} - gaussianMI: {:.3} - always 1: {:.2}'.format(epoch, outLossHat, outGausianMI, outShouldAlwaysBeOne))
#         loss1_now = sess.run(loss_hat1,feed_dict=feedDict)
#         loss2_now = sess.run(loss_hat2,feed_dict=feedDict)
#         loss1_now = np.sum(loss1_now)/trainingParam.nBatches
#         loss2_now = np.sum(loss2_now)/trainingParam.nBatches
#         alpha_loss_temp = loss1_now/(loss1_now + loss2_now)
#         # if epoch % 10 == 0:
#         #   save_data(epoch)
#   #     break
#   # break

#   save_data(i)

Each decoder as turn

In [ ]:
# 
# np_SNR_db = chParam.SNR_db * np.ones((1,1))
count = 0
for i in chParam.SNR_db:
  init = tf.global_variables_initializer()# zero gradient
  sess = tf.Session(config=config)
  sess.run(init)
  loss1_now = loss_hat1
  loss2_now = loss_hat2
  alpha_loss_temp = 0.5
  print("snr %f" % i)
  np_SNR_db = np.round(i,1) * np.ones((1,1))
  for epoch in range(1, trainingParam.iterations+1):
      if epoch % 2 == 0:

        # sess.run(resetOps1)
        sess.run(resetOps)
        # batch_progressive = [int(i) for i in np.linspace(100,10000,trainingParam.iterations)]
        # lr_progressive = [i for i in np.linspace(1e-3,1e-5,trainingParam.iterations)]
        for batch in range(0,trainingParam.nBatches):
        # for batch in range(0,batch_progressive[epoch-1]):
            # feedDict = {SNR_db: np_SNR_db,learning_rate1: lr_progressive[epoch-1]}
            feedDict = {SNR_db: np_SNR_db,learning_rate:0.001,alpha_loss: alpha_loss_temp}
            # feedDict1 = {SNR_db: np_SNR_db,learning_rate1:0.001}
            sess.run([optimizer, updateOps], feed_dict=feedDict)
            
            # sess.run([optimizer1, updateOps1], feed_dict=feedDict1)
        [outLossHat, outGausianMI, outShouldAlwaysBeOne] = sess.run([meanMetricOpsDict['loss_hat'],
                                                      meanMetricOpsDict['gaussian_MI_metric'],
                                                      should_always_be_one], feed_dict=feedDict)
        loss1_now = sess.run(loss_hat1,feed_dict=feedDict)
        loss1_now = np.sum(loss1_now)/trainingParam.nBatches
        alpha_loss_temp = loss1_now/(loss1_now + loss2_now)
        print('IQ one epoch: {} - lossHat: {:.3} - gaussianMI: {:.3} - always 1: {:.2}'.format(epoch, outLossHat, outGausianMI, outShouldAlwaysBeOne))
      else:
        # sess.run(resetOps2)
        sess.run(resetOps)
        # batch_progressive = [int(i) for i in np.linspace(100,10000,trainingParam.iterations)]
        # lr_progressive = [i for i in np.linspace(1e-3,1e-5,trainingParam.iterations)]
        for batch in range(0,trainingParam.nBatches):
        # for batch in range(0,batch_progressive[epoch-1]):
            # feedDict = {SNR_db: np_SNR_db,learning_rate2: lr_progressive[epoch-1]}
            feedDict = {SNR_db: np_SNR_db,learning_rate:0.001,alpha_loss: alpha_loss_temp}
            # feedDict2 = {SNR_db: np_SNR_db,learning_rate2:0.001}
            sess.run([optimizer, updateOps], feed_dict=feedDict)
            # sess.run([optimizer2, updateOps2], feed_dict=feedDict2)
        # outx,outy,con = sess.run([x,y,norm_constellation], feed_dict=feedDict)
        [outLossHat, outGausianMI, outShouldAlwaysBeOne] = sess.run([meanMetricOpsDict['loss_hat'],
                                                      meanMetricOpsDict['gaussian_MI_metric'],
                                                      should_always_be_one], feed_dict=feedDict)
        print('IQ two epoch: {} - lossHat: {:.3} - gaussianMI: {:.3} - always 1: {:.2}'.format(epoch, outLossHat, outGausianMI, outShouldAlwaysBeOne))
        loss2_now = sess.run(loss_hat2,feed_dict=feedDict)
        loss2_now = np.sum(loss2_now)/trainingParam.nBatches
        # if epoch % 10 == 0:
        #   save_data(epoch)
  #     break
  # break

  save_data(i)

Streaming output truncated to the last 5000 lines.
IQ one epoch: 40 - lossHat: -5.49 - gaussianMI: 13.3 - always 1: 1.0
IQ two epoch: 41 - lossHat: -5.51 - gaussianMI: 13.3 - always 1: 1.0
IQ one epoch: 42 - lossHat: -5.51 - gaussianMI: 13.2 - always 1: 1.0
IQ two epoch: 43 - lossHat: -5.52 - gaussianMI: 13.3 - always 1: 1.0
IQ one epoch: 44 - lossHat: -5.53 - gaussianMI: 13.1 - always 1: 1.0
IQ two epoch: 45 - lossHat: -5.54 - gaussianMI: 13.2 - always 1: 1.0
IQ one epoch: 46 - lossHat: -5.55 - gaussianMI: 13.1 - always 1: 1.0
IQ two epoch: 47 - lossHat: -5.56 - gaussianMI: 13.2 - always 1: 1.0
IQ one epoch: 48 - lossHat: -5.57 - gaussianMI: 13.3 - always 1: 1.0
IQ two epoch: 49 - lossHat: -5.58 - gaussianMI: 13.2 - always 1: 1.0
IQ one epoch: 50 - lossHat: -5.57 - gaussianMI: 13.1 - always 1: 1.0
IQ two epoch: 51 - lossHat: -5.57 - gaussianMI: 13.0 - always 1: 1.0
IQ one epoch: 52 - lossHat: -5.59 - gaussianMI: 13.1 - always 1: 1.0
IQ two epoch: 53 - lossHat: -5.59 - gaussianMI: 12.9

In [ ]:
# np.save('x.npy',outx)
# np.save('y.npy',outy)
# np.save('con.npy',con)

In [ ]:
# params=tf.trainable_variables()
# print("Trainable variables:------------------------")

# for idx, v in enumerate(params):
#      print("  param {:3}: {:15}   {}".format(idx, str(v.get_shape()), v))

In [ ]:
# np_x = []
# np_y = []
# for _ in range(1000):
#     temp_x, temp_y = sess.run([x, y], feed_dict=feedDict)
#     np_x.append(temp_x)
#     np_y.append(temp_y)

In [ ]:
# all_y = np.reshape(np.stack(np_y),-1)
# all_x = np.reshape(np.stack(np_x),-1)

# noise = np.random.normal(0,1,size=all_x.shape)+1j*np.random.normal(0,1,size=all_x.shape) 
# all_x = all_x + 0.05*noise

In [ ]:
# np.save('/content/drive/MyDrive/x_value_no.npy',all_x)
# np.save('/content/drive/MyDrive/y_value_no.npy',all_y)

In [ ]:
# heatmap, xedges, yedges = np.histogram2d(np.real(all_x), np.imag(all_x), bins=500)
# extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

# plt.figure(figsize=(8,8))
# plt.clf()
# plt.imshow(heatmap.T, extent=extent, origin='lower')
# plt.axis('square');

In [ ]:
def read_file(file):
    all_x = []
    for i in os.listdir(file):
        all_x.append(np.load(file+'/'+ i))
    return all_x

In [ ]:
all_x = read_file('/content/drive/MyDrive/colab/save_data/')

In [ ]:
len(all_x)

301

In [ ]:
count = np.linspace(0,30,301)

In [ ]:
k = 0
for i,j in zip(all_x,count):
    k+=1
    print(k)
    real = np.real(i)
    imag = np.imag(i)
    heatmap, xedges, yedges = np.histogram2d(real, imag, bins=500)
    # colors = ['white', 'blue','black','red','orange']

    # cmap = matplotlib .colors.ListedColormap(colors)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    size_x = (np.abs(xedges[0])+np.abs(xedges[-1]))/6
    size_y = (np.abs(yedges[0])+np.abs(yedges[-1]))/6
    # extent = [-3, 3, -3, 3]
    plt.figure(figsize=(8*size_x,8*size_y))
    # plt.title('Snr %0.2f | Epoch = %d' %(j,70))
    plt.imshow(heatmap.T,extent = extent) #cmap = cmap
    plt.axis('off')
    plt.savefig('/content/drive/MyDrive/colab/e2e_result/1t2_snr'+str(j)+'.png',bbox_inches='tight',pad_inches=0.0)
    plt.show()
    

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
snr = np.linspace(0,30,301)

In [ ]:
from PIL import Image
import imageio
import os

In [ ]:
def combine(img1_p,img2_p,save_img):
    img1 = Image.open(img1_p)
    img2 = Image.open(img2_p)
    w = int((img1.size[0] - img2.size[0])/2)
    h = int((img1.size[1] - img2.size[1])/2)
    img1.paste(img2, (w,h))
    img1.save(save_img)

In [ ]:
for i in snr:
    combine('/content/drive/MyDrive/colab/e2e_blank/snr'+ str(i) +'.png','/content/drive/MyDrive/colab/e2e_result/1t2_snr'+str(i)+'.png','/content/drive/MyDrive/colab/e2e_gif/frame_'+str(i)+'.png')

In [ ]:
img_paths = os.listdir('/content/drive/MyDrive/colab/e2e_gif')

In [ ]:
gif_images = []
for path in img_paths:
    print(path)
    gif_images.append(imageio.imread('/content/drive/MyDrive/colab/e2e_gif/'+path))

In [ ]:
imageio.mimsave("1t2.gif",gif_images,fps=10)